### IF4091 Pembelajaran Mesin
## Tugas Besar II: Feed Forward Neural Network
**Anggota Kelompok:**
1. **13515021 - Dewita Sonya T**
2. **13515057 - Erick Wijaya**
3. **13515063 - Kezia Suhendra**

**Kelas: K01**

# Import Statement

In [49]:
from copy import deepcopy
from tensorflow.keras import optimizers
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.models import Sequential

import tensorflow.keras as keras
import numpy as np
import pandas as pd

# Preprocessing Functions

In [50]:
def onehot_encode(df, colname):
    uniques = np.unique(df[colname]).tolist()
    dict_cols = {key: np.zeros((df.shape[0],), dtype=int) for key in uniques}
    for i, row in df.iterrows():
        for key in dict_cols:
            if row[colname] == key:
                dict_cols[key][i] = 1
                break
    for key, val in dict_cols.items():
        df["_" + key] = val
    return df, list(dict_cols.keys())

def label_encode(df, colname):
    uniques = np.unique(df[colname]).tolist()
    dict_unique = {key: i for i, key in enumerate(uniques)}
    rows = [dict_unique[row[colname]] for _, row in df.iterrows()]
    df["_" + colname] = np.array(rows)
    return df, dict_unique

def zscore_encode(df, colname):
    mean = np.mean(df[colname])
    std = np.std(df[colname])
    df["z_" + colname] = np.array([(row[colname]-mean)/std for _, row in df.iterrows()])
    return df, mean, std

# Preprocess

In [88]:
# Load Data
# df = pd.read_csv("iris.txt", header=None)
df = pd.read_csv("weather.txt")
print("Raw:"); display(df)

# Preprocess
df, values_outlook = onehot_encode(df, "outlook")
df, dict_windy = label_encode(df, "windy")
df, dict_play = label_encode(df, "play")
df, mean_temp, std_temp = zscore_encode(df, "temperature")
df, mean_hum, std_hum = zscore_encode(df, "humidity")
del df["outlook"], df["windy"], df["play"] , df["temperature"], df["humidity"]

# Split Data 80/10/10
train_df, validation_df, test_df = np.split(df.sample(frac=1), [int(.8*len(df)), int(.9*len(df))])
train_labels = train_df["_play"].values
validation_labels = validation_df["_play"].values
test_labels = test_df["_play"].values
del train_df["_play"], validation_df["_play"], test_df["_play"]
# train_labels = train_df[4].values
# validation_labels = validation_df[4].values
# test_labels = test_df[4].values
# del train_df[4], validation_df[4], test_df[4]
print("Train:"); display(train_df); display(train_labels)
print("Validation:"); display(validation_df); display(validation_labels)
print("Test:"); display(test_df); display(test_labels)

print(values_outlook, dict_windy, dict_play)

Raw:


,outlook,temperature,humidity,windy,play
0,sunny,85,85,False,no
1,sunny,80,90,True,no
2,overcast,83,86,False,yes
3,rainy,70,96,False,yes
4,rainy,68,80,False,yes
5,rainy,65,70,True,no
6,overcast,64,65,True,yes
7,sunny,72,95,False,no
8,sunny,69,70,False,yes
9,rainy,75,80,False,yes


Train:


,_overcast,_rainy,_sunny,_windy,z_temperature,z_humidity
8,0,0,1,0,-0.721886,-1.174731
6,1,0,0,1,-1.511449,-1.679217
3,0,1,0,0,-0.563974,1.448595
13,0,1,0,1,-0.406061,0.944109
0,0,0,1,0,1.804715,0.338726
9,0,1,0,0,0.225589,-0.165760
5,0,1,0,1,-1.353537,-1.174731
7,0,0,1,0,-0.248148,1.347697
4,0,1,0,0,-0.879799,-0.165760
1,0,0,1,1,1.015152,0.843212


array([1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1])

...

array([1])

Test:


,_overcast,_rainy,_sunny,_windy,z_temperature,z_humidity
12,1,0,0,0,1.173065,-0.670245
10,0,0,1,1,0.225589,-1.174731


array([1, 1])

['overcast', 'rainy', 'sunny'] {False: 0, True: 1} {'no': 0, 'yes': 1}


# Classifiers

## Backpropagation

In [28]:

class NNClassifier:
    def __init__(self, n_nodes=[], lrate=0.05, momentum=0, batch_size=1, max_iter=100):
        # Checking parameter input
        if (len(n_nodes) > self.__MAX_HIDDEN):
            raise ValueError('Number of hidden layers cannot be greater than {}'.format(self.__MAX_HIDDEN))

        if (not all(x > 0 for x in n_nodes)):
            raise ValueError('Number of nodes in a layer cannot be nonpositive')

        if (batch_size <= 0):
            raise ValueError('Batch size cannot be nonpositive')

        # Setting parameter
        self.n_nodes = n_nodes
        self.n_hiddens = len(n_nodes)
        self.lrate = lrate
        self.momentum = momentum
        self.batch_size = batch_size
        self.max_iter = max_iter
        self.weights = []
        self.prev_weights = []

    @property
    def __MAX_HIDDEN(self):
        return 10

    def __stochastic_gradient_descend(self, data, target):
        # Initialize error
        new_weights = deepcopy(self.weights)
        for i in range(len(new_weights)):
            for j in range(len(new_weights[i])):
                for k in range(len(new_weights[i][j])):
                    new_weights[i][j][k] = 0;
                    
        for x, y in zip(data, target):
            x = np.append(x, 1.)
            values_layers = self.__feed_forward(x)
            errors_layers = self.__backward_prop(y, values_layers)
            values_layers.insert(0, x)

            # Store weight
            for ilayer, (weights_per_layer, prev_weights_per_layer) in enumerate(zip(self.weights, self.prev_weights)):
                for inode, (weight_all, prev_weight_all) in enumerate(zip(weights_per_layer, prev_weights_per_layer)):
                    for iweight, (weight, prev_weight) in enumerate(zip(weight_all, prev_weight_all)):
                        new_weights[ilayer][inode][iweight] += self.__calculate_weight(prev_weight, 
                            values_layers[ilayer][inode], errors_layers[ilayer][iweight])
                
        # Update
        prev_weights = new_weights
        for i in range(len(new_weights)):
            for j in range(len(new_weights[i])):
                for k in range(len(new_weights[i][j])):
                    self.weights[i][j][k] += new_weights[i][j][k]
    
    def __feed_forward(self, x):
        outputs = [x]
        for weight in self.weights:
            outputs.append(self.__sigmoid(outputs[-1] @ weight))
        del outputs[0]
        return outputs
    
    def __sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def __backward_prop(self, target, values_layers):
        n_hiddens_out_layers = len(values_layers)
        errors_layers = [None] * n_hiddens_out_layers
        for i in range(n_hiddens_out_layers-1, 0-1, -1):
            errors = []
            if i < n_hiddens_out_layers-1: # (hidden layer)
                for inode, output in enumerate(values_layers[i]):
                    errors.append(self.__hidden_error(output, inode, i, errors_layers))
            else: # i == n_hiddens_out_layers-1 (output layer)
                for output in values_layers[i]:
                    errors.append(self.__output_error(output, target))
            errors_layers[i] = np.array(errors)
        return errors_layers

    def __output_error(self, output, target):
        return output * (1 - output) * (target - output)
    
    def __hidden_error(self, output, inode, index_layer, errors_layers):
        index_delta = index_layer + 1
        index_weight = index_layer + 1
        sigma = 0
        for i in range(0, len(self.weights[index_weight][inode])):
            # takut salah indexnya
            sigma += self.weights[index_weight][inode][i] * errors_layers[index_delta][i]
        return output * (1 - output) * sigma

    def __calculate_weight(self, weight, prev_weight, err, val):
        return weight + self.momentum * prev_weight + self.lrate * err * val

    def fit(self, data, target):
        self.__initialize_weights(data)

        for _ in range(self.max_iter):
            # Random shuffle data and target simultaneously
            p = np.random.permutation(data.shape[0])
            data, target = data[p], target[p]

            # Do gradient descent per batch
            for i in range(0, data.shape[0], self.batch_size):
                index = list(range(i, i+self.batch_size))
                self.__stochastic_gradient_descend(data[index], target[index])
            
        return self

    def __initialize_weights(self, data):
        # Initialize weights with random numbers
        n_features = data.shape[1]
        if (self.n_hiddens > 0):
            self.weights = [np.random.randn(n_features + 1, self.n_nodes[0])]
            for i in range(1, self.n_hiddens):
                self.weights.append(np.random.randn(self.n_nodes[i-1], self.n_nodes[i]))
            self.weights.append(np.random.randn(self.n_nodes[self.n_hiddens - 1], 1))
        else:
            self.weights = [np.random.randn(n_features + 1, 1)]
        
        # Assume first prev_weights be zeroes
        self.prev_weights = deepcopy(self.weights)
                    
    def predict(self, data):
        result = [self.__feed_forward(np.append(d, 1.))[-1][0] for d in data]
        return [1 if r >= 0.5 else 0 for r in result]

## Keras

In [79]:
class NNKeras():
    def __init__(self, nnodes_per_hidden_layer=[100], lrate=0.05, momentum=0, batch_size=1):
        self.nnodes_per_hidden_layer = nnodes_per_hidden_layer
        self.lrate = lrate
        self.momentum = momentum
        self.batch_size = batch_size
    
    def fit(self, data, labels, epochs=1):
        """data: ndarray"""
        n_rows = len(data)
        n_attr = len(data[n_rows-1])
        self.model = Sequential()
        # First Hidden Layer
        self.model.add(Dense(units=self.nnodes_per_hidden_layer[0], activation='sigmoid', input_dim=n_attr))
        # 2nd .. Last Hidden Layer
        for i in range(1, len(self.nnodes_per_hidden_layer)):
            self.model.add(Dense(units=self.nnodes_per_hidden_layer[i], activation='sigmoid'))
        # Output Layer
        self.model.add(Dense(units=1, activation='sigmoid'))
        
        sgd = optimizers.SGD(lr=self.lrate, momentum=self.momentum)
        self.model.compile(optimizer=sgd, loss='mean_squared_error')
        self.model.fit(data, labels, batch_size=self.batch_size, epochs=epochs, verbose=0)
        return self

    def evaluate(self, test_data, test_labels):
        return self.model.evaluate(test_data, test_labels, batch_size=self.batch_size)

    def predict(self, sample):
        return self.model.predict_classes(sample, batch_size=self.batch_size)

# Training (Hold-Out)

## Feed Forward Neural Network

In [30]:
# Evaluate with validation data
model_nn = NNClassifier(n_nodes=[20,10], lrate=0.1, momentum=0.1, batch_size=1, max_iter=450).fit(
    train_df.values, train_labels)

print("Test with train data:")
print("  Predicted", model_nn.predict(train_df.values))
print("  Expected", train_labels)
print("Test with validation data:")
print("  Predicted", model_nn.predict(validation_df.values))
print("  Expected", validation_labels)

Test with train data:
  Predicted [1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0]
  Expected [1 1 1 0 0 1 1 0 1 0 0]
Test with validation data:
  Predicted [1]
  Expected [1]


## Keras

In [20]:
# Evaluate with validation data
model_keras = NNKeras(nnodes_per_hidden_layer=[20,10], lrate=0.1, momentum=0.1, batch_size=1).fit(
    train_df.values, train_labels, epochs=450)
print("Test with train data:")
print("  Predicted", model_keras.predict(train_df.values).tolist())
print("  Expected", train_labels)
print("Test with validation data:")
print("  Predicted", model_keras.predict(validation_df.values).tolist())
print("  Expected", validation_labels)

Test with train data:
  Predicted [[1], [1], [1], [0], [0], [1], [1], [0], [1], [0], [0]]
  Expected [1 1 1 0 0 1 1 0 1 0 0]
Test with validation data:
  Predicted [[1]]
  Expected [1]


# Evaluation using test data

In [22]:
# merge train and validation data
train_validation_df = pd.concat([train_df, validation_df])
train_validation_labels = np.concatenate((train_labels, validation_labels))

## Backpropagation

In [31]:
# Evaluate with test data
print("Test with train+validation data:")
print("  Predicted", model_nn.predict(train_validation_df.values))
print("  Expected", train_validation_labels)
print("Test with test data:")
print("  Predicted", model_nn.predict(test_df.values))
print("  Expected", test_labels)

Test with train+validation data:
  Predicted [1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1]
  Expected [1 1 1 0 0 1 1 0 1 0 0 1]
Test with test data:
  Predicted [1, 1]
  Expected [1 1]


## Keras

In [26]:
# Evaluate with test data
model_keras = NNKeras(nnodes_per_hidden_layer=[20,10], lrate=0.1, momentum=0.1, batch_size=1).fit(
    train_validation_df.values, train_validation_labels, epochs=450)
print("Test with train+validation data:")
print("  Predicted", model_keras.predict(train_validation_df.values).tolist())
print("  Expected", train_validation_labels)
print("Test with test data:")
print("  Predicted", model_keras.predict(test_df.values).tolist())
print("  Expected", test_labels)

Test with train+validation data:
  Predicted [[1], [1], [1], [1], [0], [1], [1], [0], [1], [0], [0], [1]]
  Expected [1 1 1 0 0 1 1 0 1 0 0 1]
Test with test data:
  Predicted [[1], [1]]
  Expected [1 1]
